In [372]:
#!/usr/bin/env python3

import time
import json
import urllib3
import requests
import logging
from datetime import datetime
from opensearchpy import OpenSearch
from opensearch_helper import OSWriter
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="opensearchpy")

In [373]:
MDMSaddress = "http://100.102.4.10:8081/zonos-api"
muser, msecret = "mdm_dvc_admin", "Hb1VNBRD8WLAu27B"
url = f"{MDMSaddress}/api/1/devices"
url1 = MDMSaddress+"/api/1/devices"
url2 = MDMSaddress+"/api/1/bulk/device-profiles/metered-data/get"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [395]:
group_to_state = {
        "Bokaro": "Jharkhand",
        "Chandrapura": "Jharkhand",
        "Durgapur": "West Bengal",
        "Koderma": "Jharkhand",
        "Mejia": "West Bengal",
        "Panchet": "Jharkhand",
        "Raghunathpur": "West Bengal"
    }
def split_group(group):
    lstring = group.split()
    #state = None  # Ensure state is always defined

    #Finding Consumer type
    if "domestic" in group.lower():
        consumerType = "Domestic"
    elif "commercial" in group.lower():
        consumerType = "Commercial"
    else:
        consumerType = "Unknown"

    #Finding Pay Type
    if "prepaid" in group.lower():
        payType = "Prepaid"
    elif "postpaid" in group.lower():
        payType = "Postpaid"
    else:
        payType = "Unknown"
    #Finding State
    if lstring[0] == "Maithon" or lstring[0] == "Maithon/GOMDs":
        if "JH" in lstring:
            state = "Jharkhand"
        elif "WB" in lstring:
            state = "West Bengal"
        else:
            state = None  # State remains None if neither JH nor WB is found
    else:
        state = group_to_state.get(lstring[0])

    items_to_remove = ["Commercial", "Domestic", "Prepaid", "Postpaid", "JH", "WB", "prepaid", "postpaid", "domestic", "commercial"]
    group_name = ' '.join([word for word in lstring if word not in items_to_remove])
    result = {"groupName": group_name, "state": state, "consumerType": consumerType, "payType": payType}
    return result

In [396]:
def get_devices():
    notOver = True
    while notOver:
        try:
            #logger.debug("Sending request to MDM API: %s", url1)
            r = requests.get(url1, verify=False, auth=(muser, msecret))
            r.raise_for_status()
            #logger.info("Successfully fetched device data from MDM API")
            notOver = False
        except requests.exceptions.HTTPError as errh:
            #logger.error("HTTP Error: %s", errh)
            continue
        except requests.exceptions.ConnectionError as errc:
            #logger.error("Connection Error: %s", errc)
            continue
        except requests.exceptions.Timeout as errt:
            #logger.error("Timeout Error: %s", errt)
            continue
        except requests.exceptions.RequestException as err:
            #logger.error("Other Error: %s", err)
            continue
    devList = json.loads(r.content.decode('utf-8'))
#    logger.debug("Received device list: %s", devList)
    return devList

In [ ]:
devices = get_devices()

In [397]:
devices_v2 = []
for item in devices:
    result = split_group(item["groupName"])
    # Handle lastConnection safely (epoch seconds)
    last_connection = item["lastConnection"] if item["lastConnection"] is not None else 0
    delta = int(time.time()) - last_connection
    devices_v2.append({
        "deviceId": item["id"],
        "group": result
    })

In [398]:
for item in devices_v2[:1]:
    print(item)

{'deviceId': 'ISKIE077058', 'group': {'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Commercial', 'payType': 'Prepaid'}}


In [ ]:
ALLOWED_REGS = {
    "1-0:2.8.0*255",
    "1-0:10.8.0*255",
    "1-0:1.8.0*255",
    "1-0:9.8.0*255"
}
fromTime, toTime, msmtTime = [
    (datetime.utcnow() - timedelta(days=1)).strftime(f"%Y-%m-%dT{t}:00Z")
    for t in ("18:29", "18:31", "18:30")
]


Frome time:  2025-09-17T18:29:00Z
To time:  2025-09-17T18:31:00Z
MeasuredaAt:  2025-09-17T18:30:00Z


In [403]:
batchSize = 2000
count = 1
postData = []
batchDeviceList = []
batchGroupList = []
deviceList = []
groupList = []
for item in devices_v2:
    deviceList.append(item['deviceId'])
    groupList.append(item["group"])
# Accumulator for all returned rows across batches
data_dictionary = []

In [ ]:
# Accumulator for all returned rows across batches
data_dictionary = []

# url2 already set above
for idx, deviceId in enumerate(deviceList):
    postData.append({"device": deviceId, "profile": "1-0:99.2.0*255", "from": fromTime, "to": toTime})
    batchDeviceList.append(deviceId)
    batchGroupList.append(groupList[idx])

    if (len(postData) == batchSize) or (count == len(devicesFiltered)):
        notOver = True
        while notOver:
            try:
                r = requests.post(url2, json=postData, verify=False, auth=(muser, msecret))
                r.raise_for_status()
                notOver = False
            except requests.exceptions.HTTPError as errh:
                continue
            except requests.exceptions.ConnectionError as errc:
                continue
            except requests.exceptions.Timeout as errt:
                continue
            except requests.exceptions.RequestException as err:
                continue

        returnJSON = json.loads(r.content.decode('utf-8'))

        # Map returned entries back to the batch device/group lists by position
        j = 0
        for entry in returnJSON:
            if entry.get("success"):
                valueList = entry.get("value", [])
                for value in valueList:
                    for registerValue in value.get("meteredValues", []):
                        reg_id = registerValue.get("registerId")
                        if not reg_id or reg_id not in ALLOWED_REGS:
                            continue
                        data_dictionary.append({
                            "deviceId": batchDeviceList[j],
                            "groupName": batchGroupList[j],
                            "registerId": registerValue.get("registerId"),  # <-- fixed name
                            "unit": registerValue.get("unit"),
                            "value": registerValue.get("value"),
                            "measuredAt": registerValue.get("measuredAt")
                        })
            j += 1

        # Reset for next batch
        postData = []
        batchDeviceList = []
        batchGroupList = []

    count += 1



In [417]:
for item in data_dictionary[:10]:
    print(item["groupName"])

{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Domestic', 'payType': 'Postpaid'}
{'groupName': 

In [421]:
for item in data_dictionary:
    group_dict = item["groupName"]

    item.update({
        "group": group_dict.get("groupName"),
        "state": group_dict.get("state"),
        "consumerType": group_dict.get("consumerType"),
        "payType": group_dict.get("payType")
    })

    item.pop("groupName", None)


In [428]:
dev_with_data = set()
data_avail = []
for item in data_dictionary:
    dev_with_data.add(item["deviceId"])
for item in devices_v2:
    isDataAvail = item["deviceId"] in dev_with_data
    group_dict = item["group"]
    data_avail.append({
        "deviceId": item["deviceId"],
        "groupName": group_dict.get("groupName"),
        "state": group_dict.get("state"),
        "consumerType": group_dict.get("consumerType"),
        "payType": group_dict.get("payType"),
        "dataAvailable": isDataAvail
    })

{'deviceId': 'ISKIE077058', 'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Commercial', 'payType': 'Prepaid', 'dataAvailable': False}
{'deviceId': 'ISKIE070642', 'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid', 'dataAvailable': True}
{'deviceId': 'ISKIE070641', 'groupName': 'Durgapur Steel TPS', 'state': 'West Bengal', 'consumerType': 'Domestic', 'payType': 'Postpaid', 'dataAvailable': True}
{'deviceId': 'ISKIE077056', 'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Domestic', 'payType': 'Postpaid', 'dataAvailable': True}
{'deviceId': 'ISKIE070696', 'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Domestic', 'payType': 'Prepaid', 'dataAvailable': False}
{'deviceId': 'ISKIE070702', 'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerType': 'Domestic', 'payType': 'Unknown', 'dataAvailable': False}
{'deviceId': 'ISKIE070683', 'groupName': 'Panchet', 'state': 'Jharkhand', 'consumerT